In [1]:
%matplotlib inline

In [2]:
from blue import Blue
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('./data/lhc_ttbar_xsec.csv')
correlations = df.loc['Correlations'].drop('Experiment')
df = df.drop('Correlations')

In [4]:
df

,Experiment,Stat,Trig,LepS,LepI,JetR,JetI,btag,Pile,JESu,...,Scale,Rad,MC+PS,PDF,Z+j,LepM,DiBos,STop,Vdm,Lumi
ATLAS,242.4,1.7,0.4,1.2,1.7,1.2,0.1,1.0,0.0,0.6,...,0.7,0.0,3.0,2.7,0.1,0.8,0.3,2.0,2.9,6.9
CMS,239.0,2.6,3.6,0.2,4.0,3.0,0.0,1.7,2.0,4.3,...,5.6,3.8,3.3,0.5,1.5,1.9,0.5,2.3,5.0,3.6


In [5]:
correlations

Stat     0.0
Trig     0.0
LepS     0.0
LepI     0.0
JetR     0.0
JetI     0.0
btag     0.0
Pile     0.0
JESu     0.0
JESi     0.0
JESc     0.5
JESf     1.0
JESb     0.0
Scale    0.5
Rad      0.0
MC+PS    0.5
PDF      1.0
Z+j      0.0
LepM     0.0
DiBos    1.0
STop     1.0
Vdm      1.0
Lumi     0.0
Name: Correlations, dtype: float64

In [6]:
comb = Blue(df, correlations)

In [7]:
res = pd.Series(
    {comb.results_column: comb.combined_result, 
     **comb.combined_uncertainties}, name='Combination',
)

In [8]:
final_df = df.append(res)
final_df['Total'] = np.sqrt((final_df.drop('Experiment', axis=1)**2).sum(axis=1))
np.round(final_df.T, 1)

,ATLAS,CMS,Combination
Experiment,242.4,239.0,241.5
Stat,1.7,2.6,1.4
Trig,0.4,3.6,1.0
LepS,1.2,0.2,0.9
LepI,1.7,4.0,1.7
JetR,1.2,3.0,1.2
JetI,0.1,0.0,0.1
btag,1.0,1.7,0.9
Pile,0.0,2.0,0.6
JESu,0.6,4.3,1.3


In [9]:
comb.weights

array([ 0.72486562,  0.27513438])

In [10]:
from scipy.stats import chi2
chi2_ndf = comb.chi2_ndf
print('chi2 =', chi2_ndf[0], ', ndf =', chi2_ndf[1], ', p-value =', chi2.sf(*chi2_ndf))

chi2 = 0.054031315728 , ndf = 1 , p-value = 0.816191335626


In [11]:
print('Correlation between ATLAS and CMS measurements = {:.1%}'.format(comb.total_correlations[0, 1]))

Correlation between ATLAS and CMS measurements = 23.5%


In [12]:
cov = comb.total_covariance

In [13]:
cov

array([[  89.   ,   30.135],
       [  30.135,  185.22 ]])

In [19]:
def chi_square_fit(res, cov):
    inv_cov = np.linalg.inv(cov)
    def func(x):
        return (res.values - x).T @ inv_cov @ (res.values - x)
    return func

In [20]:
from scipy.optimize import minimize

In [22]:
minimize(chi_square_fit(df.Experiment, cov), [0]).x

array([ 241.46454312])

In [23]:
comb.combined_result

241.46454311755085